<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/mystyle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://drive.google.com/drive/

In [ ]:
#@title ##**Check GPU** { display-mode: "form" }
!nvidia-smi

In [ ]:
#@title ##**Clone** { display-mode: "form" }
import sys
path = f"{sys.base_prefix}/local/lib/python{sys.version_info.major}.{sys.version_info.minor}/"

!pip install image-quality
!pip install lpips
!pip install gdown
!pip install codegen
!pip install astor

%cd {path}
!git clone https://github.com/google/mystyle.git

In [ ]:
#@title ##**Create folders** { display-mode: "form" }
import os

input_folder = path+'mystyle/input'
input_test_forder = path+'mystyle/input_test'

output_folder = path+'mystyle/output'
output_train_folder = path+'mystyle/output_train'
output_generate_folder = path+'mystyle/output_generate'
output_test_folder = path+'mystyle/output_test'
output_edit_folder = path+'mystyle/output_edit'

trash_folder =  path+'mystyle/trash'
model_folder =  path+'mystyle/model'

if not os.path.exists(input_folder):
    os.makedirs(input_folder)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
if not os.path.exists(trash_folder):
    os.makedirs(trash_folder)
if not os.path.exists(model_folder):
    os.makedirs(model_folder)
if not os.path.exists(output_train_folder):
    os.makedirs(output_train_folder)
if not os.path.exists(output_test_folder):
    os.makedirs(output_test_folder)
if not os.path.exists(input_test_forder):
    os.makedirs(input_test_forder)
if not os.path.exists(output_edit_folder):
    os.makedirs(output_edit_folder)

In [ ]:
#@title ##**Add path** { display-mode: "form" }
import sys
import os
module_path = '/usr/local/lib/python3.10/mystyle/third_party/stylegan2_ada_pytorch/'
if module_path not in sys.path:
    sys.path.insert(0, module_path)

if 'PYTHONPATH' in os.environ:
    paths = os.environ['PYTHONPATH'].split(':')
    if module_path not in paths:
        os.environ['PYTHONPATH'] += ':' + module_path
else:
    os.environ['PYTHONPATH'] = module_path

print('......................................PYTHONPATH:')
print(os.environ['PYTHONPATH'])
print()
print('......................................sys.path:')
for path in sys.path:
    print(path)

In [ ]:
#@title ##**Fix code** { display-mode: "form" }
import ast
import os
import astor
import _ast
path = f"{sys.base_prefix}/local/lib/python{sys.version_info.major}.{sys.version_info.minor}/"
def add_code_to_function(file_path, function_name, code_string):
    with open(file_path) as f:
        tree = ast.parse(f.read())
    for node in tree.body:
        if isinstance(node, ast.FunctionDef) and node.name == function_name:
            parsed_code = ast.parse(code_string).body
            node.body = parsed_code + node.body
            with open(file_path, 'w') as f:
                f.write(astor.to_source(tree))
            return
    raise ValueError(f"No function named {function_name} found in file {file_path}")

file_path_generate = path+'/mystyle/generate.py'
function_name_generate = 'main'
code_string_generate = f"""raw_args = ['--anchors_path', '{path}/mystyle/output_train/w/',
    '--generator_path', '{path}/mystyle/output_train/mystyle_model.pt', 
    '--output_path', '{path}/mystyle/output_generate']"""

add_code_to_function(file_path_generate, function_name_generate, code_string_generate)

file_path_test = '/usr/local/lib/python3.10/mystyle/project.py'
function_name_test = 'main'
code_string_test = f"""raw_args = ['--anchor_dir', '{path}/mystyle/output_train/w/',
    '--generator_path', '{path}/mystyle/output_train/mystyle_model.pt',
    '--images_dir', '{path}/mystyle/input_test', 
    '--output_dir', '{path}/mystyle/output_test']"""

add_code_to_function(file_path_test, function_name_test, code_string_test)


In [ ]:
#@title ##**Load models** { display-mode: "form" }
import gdown

url = 'https://drive.google.com/uc?id=1KW7bjndL3QG3sxBbZxreGHigcCCpsDgn'
model_id =  path+'mystyle/model/model_ir_se50.pth'
gdown.download(url, model_id, quiet=False)

print('model_ir_se50.pth loaded')

import requests
import bz2
import os

url = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
landmarks_model =  path+'/mystyle/model/shape_predictor_68_face_landmarks.dat'

response = requests.get(url)
with open('temp.bz2', 'wb') as f:
    f.write(response.content)

with open(landmarks_model, 'wb') as f:
    f.write(bz2.BZ2File('temp.bz2').read())

os.remove('temp.bz2')

print('shape_predictor_68_face_landmarks.dat loaded')

url = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl'
model_generator =  path+'/mystyle/model/ffhq.pkl'
gdown.download(url, model_generator, quiet=False)

print('ffhq.pkl loaded')

url = 'https://drive.google.com/uc?id=1M-hsL3W_cJKs77xM1mwq2e9-J0_m7rHP'
model_encoder =  path+'/mystyle/model/faces_w_encoder.pt'
gdown.download(url, model_encoder, quiet=False) 

print('faces_w_encoder.pt loaded')


In [ ]:
#@title ##**Load test images** { display-mode: "form" }
import zipfile

url = 'https://drive.google.com/uc?id=1a2VSSFzmgJVcchGmcD_TQ82ENxizhghz'

path_zip = path+'/mystyle/data.zip' 
if not os.path.exists(input_folder):
    os.makedirs(input_folder)

gdown.download(url, path_zip, quiet=False)

try:
    with zipfile.ZipFile(path_zip, 'r') as zip_ref:
        zip_ref.extractall(input_folder)
    print("Extraction successful")
except zipfile.BadZipFile:
    print("Error: File is not a zip file")

In [ ]:
#@title ##**Visualization (optional)** { display-mode: "form" }
import os
import matplotlib.pyplot as plt

file_list = os.listdir(input_folder)
num_cols = 5
num_images = len(file_list)
num_rows = (num_images + num_cols - 1) // num_cols
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))

for i, filename in enumerate(file_list):
    filepath = os.path.join(input_folder, filename)
    img = plt.imread(filepath)
    row_idx = i // num_cols
    col_idx = i % num_cols
    axs[row_idx, col_idx].imshow(img)
    axs[row_idx, col_idx].axis('off')

plt.show()

In [ ]:
#@title ##**1. Crop and aligning faces** { display-mode: "form" }
#from utils import id_utils, io_utils
%cd {path}mystyle
import os
os.chdir( path+'/mystyle/utils')
!python {path}mystyle/data_preprcoess/0_align_face.py \
    --images_dir {input_folder} \
    --save_dir {output_folder} \
    --trash_dir {trash_folder} \
    --min_size 200 \
    --output_size 1024 \
    --landmarks_model {landmarks_model}

output_folder_count = len(os.listdir(output_folder))

print(f'Output count {output_folder_count}')

In [ ]:
#@title ##**Visualization (optional)** { display-mode: "form" }
import os
import matplotlib.pyplot as plt

file_list = os.listdir(output_folder)
num_cols = 5
num_images = len(file_list)
num_rows = (num_images + num_cols - 1) // num_cols
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))

for i, filename in enumerate(file_list):
    filepath = os.path.join(output_folder, filename)
    img = plt.imread(filepath)
    row_idx = i // num_cols
    col_idx = i % num_cols
    axs[row_idx, col_idx].imshow(img)
    axs[row_idx, col_idx].axis('off')

plt.show()

In [ ]:
#@title ##**2. Train (☕️~30 minutes)** { display-mode: "form" }
path = f"{sys.base_prefix}/local/lib/python{sys.version_info.major}.{sys.version_info.minor}/"
print(path)
%cd {path}mystyle

#import torch_utils

#os.chdir( path+'mystyle/third_party/stylegan2_ada_pytorch/torch_utils')
#from torch_utils import persistence
!python {path}mystyle/train.py \
    --images_dir {output_folder} \
    --output_dir {output_train_folder} \
    --generator_path {path}mystyle/model/ffhq.pkl \
    --encoder_checkpoint {path}mystyle/model/faces_w_encoder.pt

In [ ]:
#@title ##**Visualization (optional)** { display-mode: "form" }


# получаем список имен файлов в каждой директории


import os
import matplotlib.pyplot as plt
from PIL import Image


# Получаем список файлов в папке input
input_files = sorted(os.listdir(output_folder))

# Для каждого файла в input_dir показываем соответствующую пару изображений
for i in range(0, len(input_files), 2):
    input_file = os.path.join(output_folder, input_files[i])
    output_file = os.path.join(output_train_folder+'/out_image', input_files[i])

    # Открываем изображения с помощью библиотеки PIL
    input_image = Image.open(input_file)
    output_image = Image.open(output_file)

    # Создаем новую фигуру с двумя подграфиками
    fig, axes = plt.subplots(nrows=1, ncols=2)

    # Показываем изображения в подграфиках
    axes[0].imshow(input_image)
    axes[1].imshow(output_image)

    # Удаляем оси на графиках
    for ax in axes:
        ax.axis("off")

    # Показываем фигуру с изображениями
    plt.show()



In [ ]:
#@title ##**Copy result to google drive (optional)** { display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive')
import shutil

source_folder = "/usr/local/lib/python3.10/mystyle/output_train"
destination_folder = "/content/drive/MyDrive/output_train"

shutil.copytree(source_folder, destination_folder)

In [ ]:
#@title ##**Generate** { display-mode: "form" }
import shutil

path = f"{sys.base_prefix}/local/lib/python{sys.version_info.major}.{sys.version_info.minor}/"
#os.chdir( path+'mystyle/')
print(path)
%cd {path}

if os.path.isdir(output_generate_folder):
    shutil.rmtree(output_generate_folder)
os.makedirs(output_generate_folder)


!python {path}mystyle/generate.py 


In [ ]:
#@title ##**Visualization (optional)** { display-mode: "form" }
import os
import matplotlib.pyplot as plt

output_out_train_folder = output_generate_folder+'/images';
print(output_out_train_folder)
file_list = os.listdir(output_out_train_folder)
num_cols = 5
num_images = len(file_list)
num_rows = (num_images + num_cols - 1) // num_cols
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 15))

for i, filename in enumerate(file_list):
    filepath = os.path.join(output_out_train_folder, filename)
    img = plt.imread(filepath)
    row_idx = i // num_cols
    col_idx = i % num_cols
    axs[row_idx, col_idx].imshow(img)
    axs[row_idx, col_idx].axis('off')

plt.show()

In [ ]:
#@title ##**Load test image** { display-mode: "form" }
import requests

url = "https://drive.google.com/u/0/uc?id=15T663A-kBXcB4MxIuQ8PKaNOTkgzaLm9&export=download"
r = requests.get(url)
with open(input_test_forder+"/test.jpg", "wb") as f:
    f.write(r.content)

In [ ]:
#@title ##**Visualization (optional)** { display-mode: "form" }
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread(input_test_forder+"/test.jpg")

plt.imshow(img)
plt.show()

In [ ]:
#@title ##**3. Test** { display-mode: "form" }

path = f"{sys.base_prefix}/local/lib/python{sys.version_info.major}.{sys.version_info.minor}/"
#os.chdir( path+'mystyle/')
print(path)
%cd {path}

if os.path.isdir(output_test_folder):
    shutil.rmtree(output_test_folder)
os.makedirs(output_test_folder)

!python {path}mystyle/project.py 

In [ ]:
#@title ##**Visualization (optional)** { display-mode: "form" }

import os
import matplotlib.pyplot as plt
from PIL import Image

input_files = sorted([f for f in os.listdir(input_test_forder) if f.endswith(('.jpg', '.png'))])

for i in range(0, len(input_files), 2):
    input_file = os.path.join(input_test_forder, input_files[i])
    output_file = os.path.join(output_test_folder+'/out_image', input_files[i])

    input_image = Image.open(input_file)
    output_image = Image.open(output_file)

    fig, axes = plt.subplots(nrows=1, ncols=2)

    axes[0].imshow(input_image)
    axes[1].imshow(output_image)
    for ax in axes:
        ax.axis("off")
    plt.show()


In [ ]:
#@title ##**Edit (don't work)** { display-mode: "form" }

path = f"{sys.base_prefix}/local/lib/python{sys.version_info.major}.{sys.version_info.minor}/"
#os.chdir( path+'mystyle/')
print(path)
%cd {path}

!python {path}mystyle/edit.py \
  --alphas_dir {output_test_folder}'/alpha' \
  --output_dir {output_edit_folder} \
  --anchor_dir {output_train_folder}'/w' \
  --generator_path {output_train_folder}'/mystyle_model.pt' \
  --editing_direction_path {output_test_folder}'/w/test.pt'